# Keywords, Parse & Export

In [286]:
import pandas as pd
%run settings/db_connect.ipynb

In [376]:
%run parsers/keywords_parser.ipynb
kw_list = parse_keywlist('data/IN/keywlist.txt')
kw_name_dict = {rec.get_name(): rec for rec in kw_list}

In [40]:
kw_df = []
kw_relations_df = []

for kw in kw_list:
    # Mogu da koristim dict umesto tuple ali izgubicu redosled a OrderDict je ruznije praviti od dict objekta za ovaj slucaj
    kw_entry = (kw.AC, 'Keyword', kw.AC, kw.get_name(), kw.get_category(), kw.get_category_name(), kw.DE, kw.SY, kw.WW)
    kw_df.append(kw_entry)

    kw_relations_df.extend( (kw.AC, p, 'hi') for p in kw.HI)
    kw_relations_df.extend( (kw.AC, go, 'map_to') for go in kw.GO)

    
kw_df = pd.DataFrame(kw_df, columns=[':ID', ':LABEL', 'ac', 'name', 'category', 'category_name', 'def', 'synonyms', 'ww'])
kw_relations_df = pd.DataFrame(kw_relations_df, columns=[':START_ID', ':END_ID', ':TYPE'])

### Export csv for Neo4j

In [41]:
kw_df.to_csv('data/neo4j/kw.csv', index=False)
kw_relations_df.to_csv('data/neo4j/kw_relations.csv')

### Export to SQL database

In [91]:
db_engine.execute(r"""

---drop table keywords, kw_parnet, kw2go

create table if not exists keywords (
    ac        text primary key not null,
    name      text not null,
    category  text,
    category_name text,
    def       text,
    synonyms  text[],
    ww        text[]
);

create table if not exists kw_parent (
    ac     text not null,
    parent text not null,
    primary key (ac, parent)
);

create table if not exists kw2go (
    kw text not null,
    go text not null,
    primary key (kw, go)
);

""" )

In [85]:
# save keywords relation
kw_df[kw_df.columns[2:]].to_sql('keywords', db_engine,  index=False, if_exists="append")

In [102]:
# save kw_parent relation
kw_parent_df = kw_relations_df[kw_relations_df[":TYPE"] == 'hi'] [[":START_ID", ":END_ID"]]
kw_parent_df.columns = ['ac', 'parent']
kw_parent_df.to_sql('kw_parent', db_engine,  index=False, if_exists="append")

In [103]:
# save kw2go relation
kw_parent_df = kw_relations_df[kw_relations_df[":TYPE"] == 'map_to'] [[":START_ID", ":END_ID"]]
kw_parent_df.columns = ['kw', 'go']
kw_parent_df.to_sql('kw2go', db_engine,  index=False, if_exists="append")

# GO, Parse & Export

In [107]:
import json
# treba mi parser jer sadrzi Term definiciju
%run parsers/obo_parser.py 

In [ ]:
with open("data/IN/go.json", "r") as obo_file:
    term_list = [Term(t['id'], t) for t in json.load(obo_file)]

In [230]:
term_df = []
term_relations_df = []

existing_go_set = set(t.id for t in term_list)

for t in term_list:
    # Mogu da koristim dict umesto tuple ali izgubicu redosled a OrderDict je ruznije praviti od dict objekta za ovaj slucaj
    term_entry = (t.id, 'Term', t.namespace_short(), t.id, t.name, t.namespace_short(), t.is_obsolete, t.comment, t.definition[0], t.xref)
    term_df.append(term_entry)

    if t.replaced_by in existing_go_set:
        term_relations_df.append( (t.id, t.replaced_by, 'replaced_by') )
    term_relations_df.extend( (t.id, go, rt) for rt in ['is_a', 'consider'] for go in t.__dict__.get(rt) or [] if go in existing_go_set)
    term_relations_df.extend( (t.id, go, rt) for rt, go in t.relationship if go in existing_go_set)
    
term_df = pd.DataFrame(term_df, columns=[':ID', ':LABEL', ':LABEL', 'id', 'name', 'namespace', 'is_obsolete:boolean',
                                         'comment', 'def', 'xref'] )
term_relations_df = pd.DataFrame(term_relations_df, columns=[':START_ID', ':END_ID', ':TYPE'])

In [409]:
term_set = set(term.id for term in term_list)

### Export csv for Neo4j

In [119]:
term_df.to_csv('data/neo4j/term.csv', index=None)
term_relations_df.to_csv('data/neo4j/term_relations.csv', index=None)

### Export to SQL

In [236]:
db_engine.execute(r"""

--drop table go, go_relations;

create table if not exists go
(
  id text primary key not null,
  name text not null,
  namespace text not null,
  is_obsolete boolean not null,
  comment text,
  def text not null,
  xref text[]
);

create table if not exists go_relations (
  start_id text not null,
  end_id text not null,
  type text not null,
  primary key (start_id, end_id, type),
  foreign key (start_id) references go(id) on delete cascade,
  foreign key (end_id) references go(id) on delete cascade  
);

""")

In [141]:
tmp_df = term_df[term_df.columns[3:]].set_index('id')
tmp_df.columns = [c.split(':')[0] for c in tmp_df.columns]
tmp_df.to_sql('go', db_engine, if_exists='append')

Export relations

In [231]:
tmp_df = term_relations_df
tmp_df.columns = [c.lower()[1:] for c in tmp_df.columns]
tmp_df.to_sql("go_relations", db_engine, index=None, if_exists='append', )

# Export proteins

In [303]:
def clasify_disorder_40(raw, regions):
    return any(l >=40 for _,_,l in regions)

def to_predictions_df(raw_prediction):
    return pd.DataFrame( [ ( ac, len(raw),  clasify_disorder_40(raw, regions) )
                            for ac, raw, regions in raw_prediction ]
                         , columns=["ac", "len", "dis"]).set_index('ac')

In [338]:
import pickle

with open('data/predictions/vsl2b_valid.pickle', 'rb') as f:
    vsl2b_valid = to_predictions_df(pickle.load(f))
with open('data/predictions/vsl2b_random.pickle', 'rb') as fr:
    vsl2b_random = to_predictions_df(pickle.load(fr))
with open('data/predictions/vsl2b_uniform.pickle', 'rb') as fu:
    vsl2b_uniform = to_predictions_df(pickle.load(fu))

In [420]:
dis40 = vsl2b_valid.join(vsl2b_random.dis, rsuffix='_random')\
                   .join(vsl2b_uniform.dis, rsuffix='_uniform')
#dis40 = dis40.query('len >= 40') # jer dis40 mora da podrazumeva >= 40
dis40.head()

,len,dis,dis_random,dis_uniform
ac,,,,
A0A060X6Z0,489,True,True,False
A0A068FIK2,1033,True,True,False
A0A075F932,421,True,False,False
A0A078CGE6,1299,True,True,False
A0A086F3E3,192,False,True,False


### Save to dis40

In [419]:
dis40.to_sql("dis40", db_engine)
db_engine.execute("alter table dis40 add primary key (ac)")

## Export Prot -> KW & GO 

In [258]:
from Bio import SeqIO

In [259]:
with open("./data/IN/valid_proteins.fasta") as handle:
    valid_cafa_seq = list( SeqIO.parse(handle, 'fasta') )
    valid_cafa_ac = {seq.id for seq in valid_cafa_seq}

In [260]:
with open("./data/IN/uniprot/uniprot_sprot.dat") as handle:
    swiss_seq = [ rec for rec in SeqIO.parse(handle, "swiss") if
                    any(ac in valid_cafa_ac for ac in rec.annotations["accessions"]) ]


In [261]:
len(swiss_seq), len(valid_cafa_ac)

(66584, 66599)

In [390]:
db_engine.execute(r"""
create table if not exists prot2kw (
  prot text not null,
  kw text not null,

  primary key (prot, kw),
  foreign key (prot) references dis40(ac) on delete cascade,
  foreign key (kw) references keywords(ac) on delete cascade  
);
""")

In [388]:
prot2kw = [ (rec.id, kw_name_dict[kw].AC) for 
             rec in swiss_seq for kw  in rec.annotations.get('keywords', []) if rec.id in valid_cafa_ac]               
prot2kw = pd.DataFrame(prot2kw, columns=['prot', 'kw']).set_index(['prot', 'kw'])

In [391]:
prot2kw.to_sql("prot2kw", db_engine, if_exists='append')

***

In [393]:
db_engine.execute(r"""
create table if not exists prot2go (
  prot text not null,
  go text not null,

  primary key (prot, go),
  foreign key (prot) references dis40(ac) on delete cascade,
  foreign key (go) references go(id) on delete cascade  
);
""")

In [412]:
prot2go = [ (rec.id, go[3:]) for
           rec in swiss_seq for go  in rec.dbxrefs 
           if go.startswith("GO:") and rec.id in valid_cafa_ac and go[3:] in term_set ]               
prot2go = pd.DataFrame(prot2go, columns=['prot', 'go'])
prot2go.head()

,prot,go
0,P0DJZ0,GO:0030430
1,P0DJZ0,GO:0039526
2,P32234,GO:0005525
3,P83010,GO:0005783
4,P83010,GO:0043231


In [414]:
prot2go.to_csv("data/IN/prot2go.csv", index=False)

db_engine.execute(r"""
COPY prot2go FROM '/home/goksi/Desktop/Projects/master/data/IN/prot2go.csv' DELIMITER ',' CSV HEADER;
""")

%rm data/IN/prot2go.csv
